In [8]:
import pandas as pd

In [9]:
import geopandas as gp

In [10]:
from code.misc.new_buffers import update_buff

In [11]:
orgbuff = gp.read_file('../data/real/refined/buffers.geojson').set_index('')

In [12]:
orgbuff.head(3)

,geometry,id,office_id,priority,type
0,"POLYGON ((37.207659 55.99689, 37.2079215790266...",0,2378,None,foot
1,"POLYGON ((37.210838 55.99686, 37.2111765950794...",1,2378,None,stepless
2,"POLYGON ((37.145757 55.976658, 37.145594535204...",2,2387,None,foot


In [36]:
buff = gp.read_file('../code/buff_log.geojson').set_index(['type', 'office_id'])
buff.head(3)

geometry  id priority
type office_id                                                                
foot 1          POLYGON ((417146.791324592 6181035.453608866, ...  52     None
     2          POLYGON ((422274.1503604013 6167477.964874175,...  54     None
     3          POLYGON ((407207.9017504905 6186565.982379913,...  56     None

In [20]:
idx = pd.IndexSlice

In [23]:
bid = 769

In [24]:
slctd_foot = buff.loc[idx['foot', bid], 'geometry']
slctd_step = buff.loc[idx['stepless', bid], 'geometry']
buff = buff[buff.index.get_level_values(1) != bid]

In [25]:
slctd_step

type      office_id
stepless  769          POLYGON ((402878.3518886387 6182768.705627556,...
Name: geometry, dtype: object

In [29]:
buff.loc[idx['foot_to_step', :], 'geometry'].shape

(83,)

In [38]:
def get_fc(buff, slctd_foot):
    '''adds a third type of buffer
    one where foot distance is already covered,
    but stepless is not. pois and regions here
    will add a score=difference between foot and stepless

    Args:
        buff: current buffs
        slctd_step: stepless buffer for selected office
    Returns:
        buff: buff with new buffers adder
    '''
    fs = buff.loc[idx['stepless', :], :].copy()

    fs.loc[:, 'geometry'] = fs.loc[:, 'geometry'].intersection(slctd_foot)
    fs.index = pd.MultiIndex.from_tuples(
        [('foot_to_step', i) for _, i in fs.index.tolist()])
    fs = fs[~fs['geometry'].is_empty]

    return pd.concat([buff, fs]).sort_index()

def update_buff(buff, bid):
    '''update buff by removing
    and substracting
    buffers for chosen office

    Args:
        buff: buffers
        bid(int): id of chosen office
    '''

    slctd_foot = buff.loc[idx['foot', bid], 'geometry']
    slctd_step = buff.loc[idx['stepless', bid], 'geometry']
    buff = buff[buff.index.get_level_values(1) != bid]

    # normal reduction
    if 'foot' in buff.index.get_level_values(0):
        tmp = buff.loc[
            idx['foot', :], 'geometry'].difference(slctd_foot).difference(slctd_step)

        buff.loc[idx['foot', :], 'geometry'] = tmp[~tmp.is_empty]
            

    if 'stepless' in buff.index.get_level_values(0):
        tmp =  buff.loc[ idx['stepless', :], 'geometry'].difference(slctd_step)
        buff.loc[idx['stepless', :], 'geometry'] = tmp[~tmp.is_empty]
    
  
    if 'foot_to_step' in buff.index.get_level_values(0):
        tmp = buff.loc[
            idx['foot_to_step', :], 'geometry'].difference(slctd_step)

        buff.loc[idx['foot_to_step', :], 'geometry'] = tmp[~tmp.is_empty]
 
    buff = buff[~buff['geometry'].is_empty]
    buff2 = get_fc(buff, slctd_foot)

    return buff2

In [39]:
b2 = update_buff(buff, 769)